# ig_analysis

Read in the datasets created

In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
load_dotenv()

dir_name = r'../data/{}'.format(os.getenv('FOLDER_NAME'))

# The scrapper seems to have some issues, and as a result some rows have more/less items than they should, thus the usage of 'error_bad_lines=False'
followers_df = pd.read_csv('{}/followers.csv'.format(dir_name), error_bad_lines=False)
following_df = pd.read_csv('{}/following.csv'.format(dir_name), error_bad_lines=False)
feed_df = pd.read_csv('{}/feed.csv'.format(dir_name), error_bad_lines=False)

## Analyzing the 'followers.csv' and 'following.csv' files

Let's take a look at the ``followers.csv`` file structure (which is the same as the ``following.csv``):

In [ ]:
followers_df.head()

### How many followers does the user have?

In [ ]:
print('The user has {} followers'.format(len(followers_df)))

### How many accounts don't follow-back the user?

In [ ]:
no_follow_back = set(following_df['username'].values) - set(followers_df['username'].values)
print('{} account(s) do not follow the user back'.format(len(no_follow_back)))
print('The no follow-back accounts are: \n{}'.format(', '.join(list(no_follow_back))))